In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext

sc = SparkContext("local[*]", "rating")

In [25]:
import datetime
# datetime.datetime.fromtimestamp(timestamp)
ratings = [
('1','1','4.0','964982703'),
('1','3','4.0','964981247'),
('1','3','3.0','964981247')
]

rdd = sc.parallelize(ratings)

result = rdd.map ( lambda t: ( int (t[0]), int(t[1]), float(t[2]),  datetime.datetime.fromtimestamp( int(t[3])) ) )\
            .map (lambda t: (t[1], t)  )\
            .map (lambda t: (t[0], t[1][2])) # pick rating as value, movieId as key
print(result.take(2))

seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))


#[(1, 4.0), (3, 4.0)]
#[(1, (4.0, 1)), (3, (7.0, 2))]

aggRdd = result.aggregateByKey((0, 0), seqOp, combOp)

print(aggRdd.collect())

# we need avg
aggRdd.map ( lambda t: (t[0], t[1][0]/t[1][1])).collect()

# aggRdd.join(movieRdd)


[(1, 4.0), (3, 4.0)]
[(1, (4.0, 1)), (3, (7.0, 2))]


[(1, 4.0), (3, 3.5)]

In [6]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
sc.parallelize([1, 2, 3, 4]).aggregate((0, 0), seqOp, combOp)

(10, 4)

In [7]:
sc.parallelize([10,20,30]).aggregate((0, 0), seqOp, combOp)

(60, 3)

In [19]:
result.aggregateByKey((0, 0), seqOp, combOp).collect()

[(1, (4.0, 1)), (3, (4.0, 1))]

In [16]:
rdd = sc.parallelize([('a',7),('a',2),('b',2)])
rdd.aggregateByKey((0,0),seqOp,combOp)\
 .collect()

[('a', (9, 2)), ('b', (2, 1))]